In [4]:
!pip install tensorflow tensorflow-gpu pandas matplotlib scikit-learn

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  ERROR: Command errored out with exit status 1:
   command: /home/sahilsssingh5/anaconda3/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-p39ouflb/tensorflow-gpu_8efc047f0f47494cbbfd6d055caf67c3/setup.py'"'"'; __file__='"'"'/tmp/pip-install-p39ouflb/tensorflow-gpu_8efc047f0f47494cbbfd6d055caf67c3/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-ffbskusw
       cwd: /tmp/pip-install-p39ouflb/tensorflow-gpu_8efc047f0f47494cbbfd6d055caf67c3/
  Complete output (17 lines):
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "/tmp/pip-install-p39ouflb/tensorflow-gpu_8efc047f0f47494cbbfd6d055caf67c3/setup.py"

In [3]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [4]:
df = pd.read_csv(os.path.join('jigsaw-toxic-comment-classification-challenge','train.csv','train.csv'))

In [5]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       2.1.0
aiofiles                      23.2.1
aiohttp                       3.8.1
aiosignal                     1.2.0
alabaster                     0.7.12
altair                        5.3.0
anaconda-client               1.9.0
anaconda-navigator            2.1.4
anaconda-project              0.10.2
annotated-types               0.6.0
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.6.6
astropy                       5.0.4
asttokens                     2.0.5
astunparse                    1.6.3
async-timeout                 4.0.1
atomicwrites                  1.4.0
attrs                         21.4.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1
back

In [9]:
from tensorflow.keras.layers import TextVectorization

In [10]:
x = df['comment_text']
y = df[df.columns[2:]].values

In [11]:
MAX_FEATURES = 200000

In [12]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [13]:
vectorizer.adapt(x.values)
vectorized_text = vectorizer(x.values)

In [14]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [15]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [16]:
len(dataset)

9974

In [17]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dropout,Bidirectional,Dense,Embedding

In [19]:
model = Sequential()

In [20]:
model.add(Embedding(MAX_FEATURES+1, 32))

model.add(Bidirectional(LSTM(32 , activation = 'tanh')))

model.add(Dense(128,activation = 'relu'))
model.add(Dense(256,activation = 'relu'))
model.add(Dense(128,activation = 'relu'))

model.add(Dense(6,activation = 'sigmoid'))

In [26]:
model.compile(loss='binary_crossentropy', optimizer='Adam')

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [28]:
history = model.fit(train, epochs=1, validation_data=val)

6981/6981 [==============================] - 4268s 611ms/step - loss: 0.0635 - val_loss: 0.0466


In [28]:
input_text = vectorizer('You freaking suck! I am going to hit you.')

In [29]:
res = model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 1s 517ms/step


In [30]:
(res > 0.5).astype(int)

array([[1, 1, 1, 1, 1, 0]])

In [21]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [41]:
!pip install gradio jinja2

In [42]:
pip install fsspec==2022.10.0

  Using cached fsspec-2022.10.0-py3-none-any.whl (138 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.3.1
    Uninstalling fsspec-2024.3.1:
      Successfully uninstalled fsspec-2024.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
huggingface-hub 0.22.2 requires fsspec>=2023.5.0, but you have fsspec 2022.10.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [26]:
import tensorflow as tf
import gradio as gr

In [27]:
model.save('toxicity.h5')

/home/sahilsssingh5/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
model = tf.keras.models.load_model('toxicity.h5')

In [29]:
input_str = vectorizer('hey i freaken hate you!')

In [30]:
res = model.predict(np.expand_dims(input_str,0))

1/1 [==============================] - 0s 432ms/step


In [31]:
res

array([[0.49849972, 0.4990754 , 0.50103015, 0.49758223, 0.5016988 ,
        0.503408  ]], dtype=float32)

In [32]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [33]:
try:
    interface = gr.Interface(fn=score_comment, 
                             inputs=gr.Textbox(lines=2, placeholder='Comment to score'),
                             outputs='text')
    interface.launch()
except Exception as e:
    print("An error occurred:", e)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.28.3, however version 4.29.0 is available, please upgrade.
--------


In [34]:
interface.launch(share=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Running on public URL: https://817ba4d97765c2bd28.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


ReadTimeout: The read operation timed out